# 第7章 函数装饰器和闭包

## 7.1 装饰器基础知识
装饰器是可调用的对象, 其参数是另一个函数

In [5]:
# 定义装饰器函数
def deco(func):
    print("excute deco")
    def inner():
        print("running innner")
    return inner

In [6]:
@deco  # 定义装饰器, 等于target = deco(target)
def target():
    print("running target")

excuete deco


In [3]:
target()  # 这个函数其实是inner

running innner


In [4]:
target

<function __main__.deco.<locals>.inner()>

## 7.2 Python何时执行装饰器
装饰器的一个关键特性就是, 它们在被装饰的函数定义之后立即运行。

看上面的例子excute deco立即运行。

## 7.3 使用装饰器改进“策略”模式

In [9]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func  # 使用装饰器直接注册函数

@promotion
def fidelity(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

## 7.4 变量作用域规则

In [10]:
def f1(a):
    print(a)
    print(b)

In [11]:
f1(3)

3


NameError: name 'b' is not defined

In [12]:
b = 6
f1(3)

3
6


In [13]:
# 看下面这个报错的例子
def f2(a):
    print(a)
    print(b)
    b = 9

In [14]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

Python不要求申明变量, 但是假定在函数定义中赋值的变量是局部变量。

In [15]:
b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [16]:
f3(3)

3
6


In [18]:
b  # 全局变量b被改变了s

9

## 7.5 闭包

In [19]:
# 下面利用闭包定义一个移动平均类

In [40]:
def make_average():
    series = []
    print("outer:", id(series))
    
    def averager(new_value):
        series.append(new_value)
        print("inner:", id(series))
        return sum(series) / len(series)

    return averager

In [41]:
avg1 = make_average()

outer: 139919029920968


In [42]:
avg1(10)

inner: 139919029920968


10.0

In [43]:
avg1(11)

inner: 139919029920968


10.5

In [44]:
avg2 = make_average()

outer: 139919030182664


In [45]:
avg2(10)

inner: 139919030182664


10.0

In [46]:
avg2(11)

inner: 139919030182664


10.5

In [30]:
avg1.__code__.co_varnames

('new_value',)

In [31]:
avg1.__code__.co_freevars  # 自由变量

('series',)

In [35]:
for i in avg1.__closure__:
    print(i.cell_contents)  # 这里面保存了自由变量的值

[10, 11]


## 7.6 nonlocal声明

In [36]:
# 下面看这样一个错误的例子
def make_average():
    total = 0
    count = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [37]:
avg = make_average()

In [38]:
avg(10)  # Python在averager函数中自动将count变量视作local的了（因为count和total是不可变类型, +=会改变内存地址, 导致Python觉得这是local）

UnboundLocalError: local variable 'count' referenced before assignment

In [47]:
# 下面看怎么利用nonlocal解决这个问题
def make_average():
    total = 0
    count = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [48]:
avg = make_average()

In [49]:
avg(10)

10.0

In [50]:
avg(4)

7.0

## 7.7 实现一个简单的装饰器

In [89]:
import time
# 函数计时器
def clock(func):
    
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ",".join(repr(arg) for arg in args)
        print("[{:.8f}s]{}({}) ---- >  {}".format(elapsed, name, arg_str, result))
        return result
    
    return clocked

In [90]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
    
@clock
def factorial(n, name):
    return 1 if n < 2 else n*factorial(n-1, name)

In [91]:
factorial.__name__

'clocked'

In [92]:
factorial.__doc__

In [93]:
snooze(0.123)

[0.12570221s]snooze(0.123) ---- >  None


In [94]:
factorial(5, name = "ss")

[0.00000081s]factorial(1,'ss') ---- >  1
[0.00078963s]factorial(2,'ss') ---- >  2
[0.00089795s]factorial(3,'ss') ---- >  6
[0.00099848s]factorial(4,'ss') ---- >  24
[0.00112627s]factorial(5) ---- >  120


120

In [ ]:
################################ 自定义wrap begin ###################################################

从装饰器的原理分析可以知道其实snooze和factorial两个函数都是clocked函数了, 如果想要保留原函数的__name__和__doc__的话就需要用functool.wrap

In [95]:
import time
import functools
# 函数计时器
def clock(func):
    
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ",".join(repr(arg) for arg in args)
        print("[{:.8f}s]{}({}) ---- >  {}".format(elapsed, name, arg_str, result))
        return result
    
    return clocked

In [96]:
@clock
def factorial(n, name):
    return 1 if n < 2 else n*factorial(n-1, name)

In [97]:
factorial.__name__

'factorial'

In [98]:
factorial.__doc__

下面自己实现一个functools.wraps

In [112]:
def my_wraps(func_copy):
    def wraps(func_wrap):
        """拷贝函数的__name__和__doc__
        """

        def wrap(*args, **kwargs):
            func_wrap(*args, *kwargs)

        wrap.__name__ = func_copy.__name__
        wrap.__doc__ = func_copy.__doc__
        return wrap
    return wraps

In [113]:
# 定义一个装饰器
def outter(func):
    
    def inner(*agrs, **kwargs):
        print("inner")
        func(*agrs, **kwargs)
    return inner

In [114]:
@outter
def f1(name, age):
    print("name: {}, age: {}".format(name, age))

In [115]:
f1.__name__

'inner'

In [118]:
def outter2(func):
    
    @my_wraps(func)
    def inner2(*agrs, **kwargs):
        print("inner")
        func(*agrs, **kwargs)
    return inner2

In [119]:
@outter2
def f2(name, age):
    print("name: {}, age: {}".format(name, age))

In [120]:
f2.__name__

'f2'

In [122]:
################################ 自定义wrap end ###################################################

## 7.8 标准库中的装饰器

### 7.8.1 使用functools.lru_cache做备忘
这是一个非常实用的装饰器, 他实现了备忘的功能。这是一个优化技术，他把耗时的函数结果保存下来，避免传入相同参数时进行重复计算。

Least Recently Used

In [123]:
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [124]:
fibonacci(6)

[0.00000073s]fibonacci(0) ---- >  0
[0.00000087s]fibonacci(1) ---- >  1
[0.00095879s]fibonacci(2) ---- >  1
[0.00000046s]fibonacci(1) ---- >  1
[0.00000071s]fibonacci(0) ---- >  0
[0.00000060s]fibonacci(1) ---- >  1
[0.00019836s]fibonacci(2) ---- >  1
[0.00042954s]fibonacci(3) ---- >  2
[0.00167107s]fibonacci(4) ---- >  3
[0.00000038s]fibonacci(1) ---- >  1
[0.00000045s]fibonacci(0) ---- >  0
[0.00000064s]fibonacci(1) ---- >  1
[0.00019921s]fibonacci(2) ---- >  1
[0.00048926s]fibonacci(3) ---- >  2
[0.00000039s]fibonacci(0) ---- >  0
[0.00000059s]fibonacci(1) ---- >  1
[0.00020101s]fibonacci(2) ---- >  1
[0.00000035s]fibonacci(1) ---- >  1
[0.00000061s]fibonacci(0) ---- >  0
[0.00000032s]fibonacci(1) ---- >  1
[0.00002647s]fibonacci(2) ---- >  1
[0.00005495s]fibonacci(3) ---- >  2
[0.00035425s]fibonacci(4) ---- >  3
[0.00104735s]fibonacci(5) ---- >  5
[0.00283460s]fibonacci(6) ---- >  8


8

In [125]:
# 上面这个函数在递归重复计算很多已经计算过的值, 所以这里使用缓存把结果存起来

In [126]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [127]:
fibonacci(6)

[0.00000070s]fibonacci(0) ---- >  0
[0.00000071s]fibonacci(1) ---- >  1
[0.00097689s]fibonacci(2) ---- >  1
[0.00000128s]fibonacci(3) ---- >  2
[0.00118204s]fibonacci(4) ---- >  3
[0.00000124s]fibonacci(5) ---- >  5
[0.00148871s]fibonacci(6) ---- >  8


8

注意:@functools.lru_cache(maxsize = 128, typed = False) 可以接受这两个参数。前者指定多少个保存结果，后者指定是否把不同的结果类型分开储存。

### 7.8.2 单分派 泛函数
由于Python是不支持函数的重载的, 所以如果根据类型区分的话需要很多if else, 这样代码会很耦合。

In [134]:
from functools import singledispatch
from collections import abc
import numbers
import html

In [135]:
@singledispatch  # 定义一个分配函数
def htmlize(obj):
    content = html.escape(repre(obj))
    return "<pre>{}</pre>".format(content)

In [137]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace("\n", "<br>\n")
    return "<p>{0}</p>".format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return "<pre>{0} (0x{0:x})</pre>".format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)  # 多个注册嵌套
def _(seq):
    return "seq"

In [138]:
htmlize(16)

'<pre>16 (0x10)</pre>'

In [139]:
htmlize("ss")

'<p>ss</p>'

## 7.9 叠放装饰器
将装饰器叠置起来原理很简单, 这里就不再说了

## 7.10 参数化装饰器
为了让一个装饰器能够接受参数, 我们就需要用到装饰器的工厂函数

### 7.10.1 一个参数化的注册装饰器

In [141]:
registry = set()

def register(active=True):
    def decorate(func):
        print("running register(active={})->decorate({})".format(active, func.__name__))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

In [142]:
@register(active=False)
def f1():
    print("running f1()")

running register(active=False)->decorate(f1)


In [147]:
@register()
def f2():
    print("running f2()")

running register(active=True)->decorate(f2)


In [149]:
registry

{<function __main__.f2()>, <function __main__.f2()>}

工业级装饰器最好通过实现\_\_call\_\_的类实现